# Buildkite API development

In [ ]:
!cd ..

In [ ]:
import dotenv

In [ ]:
dotenv.load_dotenv()

In [ ]:
import os

token = os.getenv("BUILDKITE_PAT")

In [ ]:
import requests

base_url = "https://api.buildkite.com/v2"

h = {"Authorization": f"Bearer {token}"}

In [ ]:
pipelines = requests.get(base_url + "/organizations/clima/pipelines", headers=h).json()

for pl in pipelines:
    print(pl["slug"])

In [ ]:
builds = requests.get(
    base_url + "/organizations/clima/pipelines/climacoupler-coarse-nightly-amip/builds",
    headers=h,
    params={"created_from": "2026-01-15"},
).json()

for b in builds:
    print(b["id"])